In [45]:
import re
from typing import List, Tuple, Dict

def parse_alpha_geometry(input_text: str) -> Tuple[str, str, str]:
    """Split AlphaGeometry input into premises, auxiliary constructions, and proof steps."""
    sections = re.split(r"\*\s+", input_text)
    premises = sections[1].strip() if len(sections) > 1 else ""
    auxiliary = sections[2].strip() if len(sections) > 2 else ""
    proof = sections[3].strip() if len(sections) > 3 else ""
    return premises, auxiliary, proof

def extract_auxiliary_constructions(aux_text: str) -> List[str]:
    """Extract point construction instructions from the auxiliary section."""
    constructions = []
    lines = aux_text.splitlines()
    for line in lines:
        match = re.match(r"([A-Z])\s*:\s*Points", line.strip())
        if match:
            point = match.group(1)
            constructions.append(f"Construct point {point}")
    return constructions

def extract_constructions_from_premises(premise_text: str) -> List[str]:
    """Extract geometric configurations from the premises section."""
    constructions = []
    lines = premise_text.strip().splitlines()
    for line in lines:
        line = line.strip()
        if 'are collinear' in line:
            pts = re.findall(r"\b[A-Z]\b", line)
            if pts:
                constructions.append(f"Points {'-'.join(pts)} are collinear")
        elif '∥' in line:
            pts = re.findall(r"\b[A-Z]{2}\b", line)
            if len(pts) == 2:
                constructions.append(f"Lines {pts[0]} ∥ {pts[1]}")
    return constructions

def extract_proof_steps(proof_text: str) -> Tuple[List[str], List[Tuple[int, List[str], List[List[str]]]]]:
    """Extract proof steps and highlight information, including used segments."""
    steps = []
    hl_data = []
    lines = proof_text.strip().splitlines()

    for idx, line in enumerate(lines):
        step_text = line.strip()[5:]
        step_text = re.sub(r'\[\d+\]', '', step_text).strip()  # remove [NN]

        steps.append(step_text)

        # Collect point names
        involved_points = re.findall(r"\b[A-Z]{1,2}\b", step_text)
        single_points = sorted(set(p for p in involved_points if len(p) == 1))

        # Collect segments like CG, GB, etc.
        segments_raw = re.findall(r"\b([A-Z]{2})\b", step_text)
        segments = []
        for s in segments_raw:
            if len(s) == 2:
                a, b = s[0], s[1]
                if a != b:  # avoid AA etc.
                    segments.append([a, b])

        hl_data.append((idx + 2, single_points, segments))  # pp starts at 2

    return steps, hl_data


def extract_elements(premise_text: str, auxiliary_points: List[str]) -> List[Tuple[str, int]]:
    """Include all points: premise points at index 0, auxiliary points with actual step index — no duplicates."""
    elements = {}
    lines = premise_text.strip().splitlines()

    for idx, line in enumerate(lines):
        points = re.findall(r"\b[A-Z]\b", line)
        for p in set(points):
            if p not in elements:
                if p in auxiliary_points:
                    elements[p] = idx  # use actual line index for auxiliary point
                else:
                    elements[p] = 0    # always 0 for premise point

    return [(p, elements[p]) for p in sorted(elements)]

def generate_cindy_structures(input_text: str) -> Dict[str, object]:
    premise, auxiliary, proof = parse_alpha_geometry(input_text)
    aux_point_names = extract_auxiliary_constructions(auxiliary)
    premise_constructions = extract_constructions_from_premises(premise)
    proof_steps, hl = extract_proof_steps(proof)
    els = extract_elements(premise, aux_point_names)

    construction_list = [f"Construct point {p}" for p in aux_point_names] + premise_constructions

    return {
        "els": els,
        "hl": hl,
        "texts": proof_steps,
        "constructions_needed": construction_list
    }

def format_els(els: List[Tuple[str, int]]) -> str:
    return "[" + ", ".join(f'[{p},{i}]' for p, i in els) + "]"

def format_hl(hl: List[Tuple[int, List[str], List[List[str]]]]) -> str:
    def format_entry(entry):
        idx, points, lines = entry
        point_list = "[" + ", ".join(points) + "]"
        line_list = "[" + ", ".join("[" + ",".join(seg) + "]" for seg in lines) + "]"
        return f"[{idx-1},{point_list},{line_list}]"
    return "[\n " + ",\n ".join(format_entry(e) for e in hl) + "]"

def clean_texts(texts: List[str]) -> List[str]:
        return [re.sub(r'\[\d+\]', '', t).strip() for t in texts]


def generate_cindy_output(input_text: str) -> Tuple[List[str], str]:
    """Return (construction instructions, full CindyScript code)"""
    data = generate_cindy_structures(input_text)

    cleaned_texts = clean_texts(data["texts"])
    els_str = format_els(data["els"])
    hl_str = format_hl(data["hl"])
    texts_str = "[\n" + ''.join(f'    "{t}",\n' for t in cleaned_texts) + '    "QED"\n]'

    proof_steps = len(data["texts"])

    static_code = f"""
createPoint("Ptop",(16,{proof_steps}));
createPoint("Pbot",(16,0));
Ptop.color = gray(0.3);
Pbot.color = gray(0.3);

draw(Ptop);draw(Pbot);

o=.13;
ll=-o;
mm=1+o;

pp=round(|Ptop,Pslider|)+1;

els = {els_str};

hl = {hl_str};

texts = {texts_str};

drawext(li,co):=(
    draw([ll * li_1 + mm * li_2,
      ll * li_2 + mm * li_1],
     	size->8, color->co);
);

highlight(el,col):=(
  co=if(col==1,(1,1,0.5),(1,1,0)*.8);
  if(isline(el),
    inc=incidences(el);
    forall(pairs(inc),li,drawext(li,co));
  );
  if(ispoint(el),
    draw(el,size->10,color->co,border->false);
  );
  if(islist(el),
    drawext(el,co);
  )
);

if(length(hl) > 0,
  apply(hl,h,
    if(h_1 == pp,
      forall(h_3, highlight(#, 0));
      forall(h_2, highlight(#, 1));
    );
  );
);

apply(els,e,(e_1).visible=(e_2<pp+1));

repeat(pp-1,
  drawtext(Ptop+(1,-#+0.5),texts_#,size->14,alpha->0.5); 
);

drawtext(Ptop+(1,-pp+0.5),texts_pp,size->14);
"""
    const = ""
    const += '// Required Constructions:\n'
    for instruction in data["constructions_needed"]:
        const += f'// - {instruction}\n'
    const += '\n'
    return const + static_code.strip()






In [46]:

with open("input.txt", "r") as f:
    input_text = f.read()

output = generate_cindy_output(input_text)
print(output)


// Required Constructions:
// - Points A-E-B are collinear
// - Lines FE ∥ DB
// - Points A-D-F are collinear
// - Lines GF ∥ CD
// - Points C-A-G are collinear

createPoint("Ptop",(16,5));
createPoint("Pbot",(16,0));
Ptop.color = gray(0.3);
Pbot.color = gray(0.3);

draw(Ptop);draw(Pbot);

o=.13;
ll=-o;
mm=1+o;

pp=round(|Ptop,Pslider|)+1;

els = [[A,0], [B,0], [C,0], [D,0], [E,0], [F,0], [G,0]];

hl = [
 [1,[],[]],
 [2,[A, B, D, E, F],[[E,F], [B,D], [A,D], [D,F], [A,B], [E,B]]],
 [3,[A, C, D, F, G],[[F,G], [C,D], [C,A], [C,G], [A,D], [D,F]]],
 [4,[],[[A,D], [D,F], [A,B], [E,B], [C,A], [C,G], [A,D], [D,F], [C,A], [C,G], [A,B], [E,B]]],
 [5,[A, B, C, E, G],[[C,A], [C,G], [A,B], [E,B], [C,B], [G,E]]]];

texts = [
    "steps:",
    "EF ∥ BD  & A,D,F are collinear  & A,E,B are collinear  ⇒  AD:DF = AB:EB",
    "FG ∥ CD  & C,A,G are collinear  & A,D,F are collinear  ⇒  CA:CG = AD:DF",
    "AD:DF = AB:EB  & CA:CG = AD:DF  ⇒  CA:CG = AB:EB",
    "CA:CG = AB:EB  & C,A,G are collinear  & A,E,B 